<h1>Chapter 7 - Advanced Text Generation Techniques and Tools</h1>
<i>Going beyond prompt engineering.</i>

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter07/Chapter%207%20-%20Advanced%20Text%20Generation%20Techniques%20and%20Tools.ipynb)

---

This notebook is for Chapter 7 of the [Hands-On Large Language Models](https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961) book by [Jay Alammar](https://www.linkedin.com/in/jalammar) and [Maarten Grootendorst](https://www.linkedin.com/in/mgrootendorst/).

---

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961">
<img src="https://raw.githubusercontent.com/HandsOnLLM/Hands-On-Large-Language-Models/main/images/book_cover.png" width="350"/></a>

### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---


In [2]:
%%capture
!pip install langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.1 datasets>=2.18.0 accelerate>=0.27.2 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2 langchain_community
!CMAKE_ARGS="-DLLAMA_CUDA=on" pip install llama-cpp-python==0.2.69

# Loading an LLM

In [3]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

# If this command does not work for you, you can use the link directly to download the model
# https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf

--2025-05-11 09:23:21--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf
Resolving huggingface.co (huggingface.co)... 13.35.202.34, 13.35.202.40, 13.35.202.97, ...
Connecting to huggingface.co (huggingface.co)|13.35.202.34|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/5d99003e395775659b0dde3f941d88ff378b2837a8dc3a2ea94222ab1420fad3?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-fp16.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-fp16.gguf%22%3B&Expires=1746959001&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0Njk1OTAwMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvNWQ5OTAwM2UzOTU3NzU2NTliMGRkZTNmOTQxZDg4Z

In [4]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-fp16.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [5]:
llm.invoke("Привет! Меня зовут Александр. Сколько будет 1+1?")

'\n**Response:** The answer to the arithmetic question "Сколько будет 1+1?" is 2. This question requires basic math skills, specifically addition. In Russian, you would say "1 + 1 равно 2." Therefore, regardless of the initial context about Alexander\'s name, when asked this simple mathematical problem in Russian or any other language, the answer remains the same: 2.'

### Chains

In [6]:
from langchain import PromptTemplate

# Create a prompt template with the "input_prompt" variable
template = """<s><|user|>
{input_prompt}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt"]
)

In [7]:
basic_chain = prompt | llm

In [8]:
# Use the chain
basic_chain.invoke(
    {
        "input_prompt": "Привет! Меня зовут Александр. Какие малоизвестные, но перспективные криптовалюты ты знаешь?",
    }
)

' Привет Александр! Уровень новиznosti и перспективности криптовалют может быть сложным для определения, но вот некоторые криптовалюты, которые не так широко известны, но все же имеют потенциал:\n\n1. Tezos: Фундаментальные криптосеты, основанную на технологии обобщенной планирографии и программного обеспечения открытого исходного кода.\n2. Avalanche: Сеть децентрализованных счетов, основанная на NFT и Proof-of-Stake алгоритме.\n3. Cosmos: Сеть, использующая несколько независимых оси-системы, связанных между собой через специальные интерфейсы.\n4. Polkadot: Сочиниваatorio, позволяющее интеграцию разнородных криптосет с помощью тонких данных и обмен между ними на уровне параллельных транзакций.\n5. Solana: Сеть-диктатор, основанная на Proof-of-Stake алгоритме с высокой скоростью транзакций, благодаря использованию специального распределителя данных (SSD).\n6. Algorand: Прототип системы опенсорса для криptoсеты, использующий Proof-of-Stake алгоритм, с высокой скоростью транзакций и безоп

### Multiple Chains

In [9]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<s><|user|>
Create a title for a story about {summary}. Only return the title.<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

<ipython-input-9-61dd782c6da9>:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")


In [10]:
title.invoke({"summary": "учёный который изобретает криптовалюту будущего"})

{'summary': 'учёный который изобретает криптовалюту будущего',
 'title': ' "Genesis of Tomorrow: The Inventor\'s Chronicles in Future Cryptocurrency"'}

In [11]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title {title}. Use only two sentences.<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")

In [12]:
# Create a chain for the story using the summary, title, and character description
template = """<s><|user|>
Create a story about {summary} with the title {title}. The main charachter is: {character}. Only return the story and it cannot be longer than one paragraph<|end|>
<|assistant|>"""
story_prompt = PromptTemplate(
    template=template, input_variables=["summary", "title", "character"]
)
story = LLMChain(llm=llm, prompt=story_prompt, output_key="story")

In [13]:
# Combine all three components to create the full chain
llm_chain = title | character | story

In [14]:
llm_chain.invoke("учёный, который изобретает криптовалюту будущего")

{'summary': 'учёный, который изобретает криптовалюту будущего',
 'title': ' "Совершив Миracle: Художник-Изумышник Создает Криптовалюту Заднего Весна"',
 'character': ' The main character of "Совершив Миracle: Художник-Изумышник Создает Криптовалюту Заднего Весна" is a brilliant scientist and artist, Dr. Ivan Petrovich, whose groundbreaking invention combines cutting-edge cryptography with futuristic technology, earning him the title of mastermind behind creating digital currency during winter\'s obscurity. Driven by his insatiable curiosity for innovation and artistry, he embarks on a daring journey to reshape the financial landscape while facing both accolades and skepticism from society.',
 'story': ' In the depths of winter\'s embrace, Dr. Ivan Petrovich, a virtuoso scientist and avant-garde artist, unveiled his magnum opus: cryptovatu currency christened "Cryptonova," born from the nexus of intricate encryption techniques and futuristic algorithms. His groundbreaking creation defie

# Memory

In [15]:
# Let's give the LLM our name
basic_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})

" Hello Maarten, the answer to 1 + 1 is 2. It's a basic arithmetic operation where you add one unit to another unit."

In [16]:
# Next, we ask the LLM to reproduce the name
basic_chain.invoke({"input_prompt": "What is my name?"})

" I'm an AI and don't have the ability to know personal information about individuals unless it has been shared with me in the course of our conversation. Therefore, I can't know your name."

## ConversationBuffer

In [17]:
# Create an updated prompt template to include a chat history
template = """<s><|user|>Current conversation:{chat_history}

{input_prompt}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["input_prompt", "chat_history"]
)

In [18]:
from langchain.memory import ConversationBufferMemory

# Define the type of Memory we will use
memory = ConversationBufferMemory(memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-18-9823403f20ac>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [19]:
msg = "Привет! Меня зовут Александр. Какие экстравагантные, но перспективные криптовалюты ты знаешь?"

In [20]:
# Generate a conversation and ask a basic question
llm_chain.invoke({"input_prompt": msg})

{'input_prompt': 'Привет! Меня зовут Александр. Какие экстравагантные, но перспективные криптовалюты ты знаешь?',
 'chat_history': '',
 'text': ' Здравствуйте, Александр! Вот несколько интересных и перспективных криптовалют, которые вы можете рассмотреть:\n\n1. Algorand: Это открытая и практичная реализация Proof-of-Stake (PoS) алгоритма, известный за своим быстрым транзакционным процессом и высокой масштабируемости. Algorand использует уникальные публичные ключи для аутентификации участников системы, что делает его бесприемственно и безопасно.\n\n2. Tezos: Tezos представляет собой открытую криптовалюту с бесплатными транзакциями, основанной на своей собственной технологии Chiavelli, которая позволяет обновлять верификацию сети без необходимости переустановки базы данных. Tezos также поддерживает множественные криптосписки и платформы разработки.\n\n3. Cardano: Созданный с использованием системы Bayesian inference и Proof-of-Stake (PoS), Cardano стремится достичь высокого уровня безопа

In [22]:
# Does the LLM remember the name we gave it?
llm_chain.invoke({"input_prompt": "What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': 'Human: Привет! Меня зовут Александр. Какие экстравагантные, но перспективные криптовалюты ты знаешь?\nAI:  Здравствуйте, Александр! Вот несколько интересных и перспективных криптовалют, которые вы можете рассмотреть:\n\n1. Algorand: Это открытая и практичная реализация Proof-of-Stake (PoS) алгоритма, известный за своим быстрым транзакционным процессом и высокой масштабируемости. Algorand использует уникальные публичные ключи для аутентификации участников системы, что делает его бесприемственно и безопасно.\n\n2. Tezos: Tezos представляет собой открытую криптовалюту с бесплатными транзакциями, основанной на своей собственной технологии Chiavelli, которая позволяет обновлять верификацию сети без необходимости переустановки базы данных. Tezos также поддерживает множественные криптосписки и платформы разработки.\n\n3. Cardano: Созданный с использованием системы Bayesian inference и Proof-of-Stake (PoS), Cardano стремится достичь высоко

## ConversationBufferMemoryWindow

In [23]:
from langchain.memory import ConversationBufferWindowMemory

# Retain only the last 2 conversations in memory
memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")

# Chain the LLM, Prompt, and Memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-23-046ef635f261>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, memory_key="chat_history")


In [24]:
# Ask two questions and generate two conversations in its memory
llm_chain.invoke({"input_prompt":"Hi! My name is Maarten and I am 33 years old. What is 1 + 1?"})
llm_chain.invoke({"input_prompt":"What is 3 + 3?"})

{'input_prompt': 'What is 3 + 3?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten, it's nice to meet you! The answer to 1 + 1 is 2.\n\nHowever, if this was part of a larger conversation or context-related question, feel free to provide more information for further engagement. For now, the simple math adds up nicely as always: one plus one equals two.",
 'text': " Hello Maarten, it's nice to meet you too! The answer to 3 + 3 is 6."}

In [25]:
# Check whether it knows the name we gave it
llm_chain.invoke({"input_prompt":"What is my name?"})

{'input_prompt': 'What is my name?',
 'chat_history': "Human: Hi! My name is Maarten and I am 33 years old. What is 1 + 1?\nAI:  Hello Maarten, it's nice to meet you! The answer to 1 + 1 is 2.\n\nHowever, if this was part of a larger conversation or context-related question, feel free to provide more information for further engagement. For now, the simple math adds up nicely as always: one plus one equals two.\nHuman: What is 3 + 3?\nAI:  Hello Maarten, it's nice to meet you too! The answer to 3 + 3 is 6.",
 'text': ' Your name is Maarten, as mentioned at the beginning of our conversation.\n\nNow for the math question: 3 + 3 equals 6.'}

In [26]:
# Check whether it knows the age we gave it
llm_chain.invoke({"input_prompt":"What is my age?"})

{'input_prompt': 'What is my age?',
 'chat_history': "Human: What is 3 + 3?\nAI:  Hello Maarten, it's nice to meet you too! The answer to 3 + 3 is 6.\nHuman: What is my name?\nAI:  Your name is Maarten, as mentioned at the beginning of our conversation.\n\nNow for the math question: 3 + 3 equals 6.",
 'text': ' I\'m an AI and do not have a physical form or age. However, if you\'re referring to the age of the information I was trained on up until September 2021, it would be considered as having "no age" in the human sense since it is data-driven knowledge.\nAnswer: As an AI, I do not have an age.'}

## ConversationSummary

In [27]:
# Create a summary prompt template
summary_prompt_template = """<s><|user|>Summarize the conversations and update with the new lines.

Current summary:
{summary}

new lines of conversation:
{new_lines}

New summary:<|end|>
<|assistant|>"""
summary_prompt = PromptTemplate(
    input_variables=["new_lines", "summary"],
    template=summary_prompt_template
)

In [28]:
from langchain.memory import ConversationSummaryMemory

# Define the type of memory we will use
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    prompt=summary_prompt
)

# Chain the LLM, prompt, and memory together
llm_chain = LLMChain(
    prompt=prompt,
    llm=llm,
    memory=memory
)

<ipython-input-28-905e44fca19a>:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryMemory(


In [30]:
# Generate a conversation and ask for the name
llm_chain.invoke({"input_prompt": "Hi! My name is Maarten. What is 1 + 1?"})
llm_chain.invoke({"input_prompt": "What is my name? do not write any other information."})

{'input_prompt': 'What is my name? do not write any other information.',
 'chat_history': " Maarten introduces himself and inquires about the result of 1 + 1, which the AI correctly answers as 2 by explaining it as a simple addition operation. The AI also clarifies that personal data isn't available without current context and reminds Maarten he previously mentioned his name in this conversation.",
 'text': ' Maarten.'}

In [ ]:
# Check whether it has summarized everything thus far
llm_chain.invoke({"input_prompt": "What was the first question I asked?"})

{'input_prompt': 'What was the first question I asked?',
 'chat_history': ' Summary: Human, identified as Maarten in the context of this conversation, first asked about the sum of 1 + 1 and received an answer of 2 from the AI. Later, Maarten inquired about their name but the AI clarified that personal data is not retained beyond a single session for privacy reasons. The AI offered further assistance if needed.',
 'text': ' The first question you asked was "what\'s 1 + 1?"'}

In [31]:
# Check what the summary is thus far
memory.load_memory_variables({})

{'chat_history': " Maarten introduces himself, inquires about the result of 1 + 1 (correctly answered as a simple addition operation), and asks for his name in the conversation; the AI confirms it's Maarten without providing any additional personal information. The AI also reminds Maarten that personal data requires current context."}

# Agents

In [ ]:
import os
from langchain_openai import ChatOpenAI

# Load OpenAI's LLMs with LangChain
# os.environ["OPENAI_API_KEY"] = "MY_KEY"
# openai_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [32]:
# Create the ReAct template
react_template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(
    template=react_template,
    input_variables=["tools", "tool_names", "input", "agent_scratchpad"]
)

In [33]:
from langchain.agents import load_tools, Tool
from langchain.tools import DuckDuckGoSearchResults

# You can create the tool to pass to an agent
search = DuckDuckGoSearchResults()
search_tool = Tool(
    name="duckduck",
    description="A web search engine. Use this to as a search engine for general queries.",
    func=search.run,
)

# Prepare tools
tools = load_tools(["llm-math"], llm=llm)
tools.append(search_tool)

In [34]:
from langchain.agents import AgentExecutor, create_react_agent

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [35]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": "What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?"
    }
)



> Entering new AgentExecutor chain...
 I need to use duckduck to find the current price and then calculate the amount in EUR using the given exchange rate.
Action: duckduck
Action Input: "current MacBook Pro price in USD"snippet: The base model of the M3 Pro starts at $2,499 USD (although exclusive discounts are available in this guide) and is an ideal option for users who want a larger screen and advanced capabilities. To get the best deal on this or any other MacBook Pro, check out AppleInsider's list of the top MacBook Pro deals., title: MacBook Pro 16-inch Best Price M3 Max, 16C CPU, 40C GPU, 48GB, 1TB ..., link: https://prices.appleinsider.com/product/macbook-pro-16-inch-m3/MUW63LL/A, snippet: For a limited time, you can get the M4 MacBook Pro (Silver) for $1,373 from go-to Apple retailer, Amazon. That's $226 below the M4 MB Pro's $1,599 list price and just $1 shy of its all-time low., title: Apple's excellent M4 MacBook Pro hits all-time low price - Laptop Mag, link: https://ww

{'input': 'What is the current price of a MacBook Pro in USD? How much would it cost in EUR if the exchange rate is 0.85 EUR for 1 USD?',
 'output': 'The current price of a base model MacBook Pro M3 in USD is $2,499. At an exchange rate of 0.85 EUR for each USD, it would cost approximately €2,124.15.\n\n\n\n\nAnswer: The current price of a base model MacBook Pro M3 in USD is $2,499. At an exchange rate of 0.85 EUR for each USD, it would cost approximately €2,124.15.'}

In [36]:
msg = 'Напиши план изучения больших языковых моделей и агентов, для максимальной прокачки в этой теме.'
msg_eng = 'Write a plan for studying large language models and agents, for maximum improvement in this topic.'

In [37]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": msg
    }
)



> Entering new AgentExecutor chain...
 To create a plan for studying large language models and agents, I need to gather information on effective learning methods and resources available for this topic.
Action: duckduck
Action Input: "планы изучения больших языковых моделей и агентов"snippet: Почему Go-ассемблер и векторизация могут быть полезны: идея для ускорения. Краткий обзор и перевод исследования Large Language Model-Based Agents for Software Engineering: A Survey , которое посвящено применению интеллектуальных агентов на основе больших языковых моделей..., title: ИИ-агенты на основе больших языковых моделей для разработки: обзор, link: https://habr.com/ru/companies/bothub/articles/842816/, snippet: Агенты LLM — это автономные системы, которые используют большие языковые модели для восприятия, планирования и выполнения действий в различных средах. Они способны обрабатывать текстовую информацию, взаимодействовать с пользователями и другими агентами, а также использовать внешние и

{'input': 'Напиши план изучения больших языковых моделей и агентов, для максимальной прокачки в этой теме.',
 'output': 'To create an effective learning plan for studying large language models and agents, follow these steps: 1) Understand the basics of'}

In [38]:
# What is the Price of a MacBook Pro?
agent_executor.invoke(
    {
        "input": msg_eng
    }
)



> Entering new AgentExecutor chain...
 I need to gather information about effective study methods and resources for learning about large language models and agents.
Action: duckduck
Action Input: "best practices for studying large language models and agents"snippet: In 2016, the company revisited that article and talked about how the advent of machine learning and natural language processing could truly open up opportunities. Today, artificial intelligence ..., title: Best Practices For Consulting Firms Exploring Large Language Models, link: https://www.forbes.com/councils/forbesbusinesscouncil/2025/02/04/best-practices-for-consulting-firms-exploring-large-language-models/, snippet: Even though such metrics help evaluate LLMs, it is best to pair them with other methods, especially when the activities being handled are complex, critical and require human judgment. LLM Evaluation Benchmarks: LLM evaluation benchmarks help evaluate the large language model's performance by offering a st

{'input': 'Write a plan for studying large language models and agents, for maximum improvement in this topic.',
 'output': 'To study large language models and agents effectively, you should follow these best practices and metrics for evaluation as suggested by various sources. Begin by understanding the fundamentals of machine learning, natural language processing (NLP), and deep learning algorithms through reputable online courses or textbooks. Utilize resources such as Forbes Business Council\'s article on "Best Practices For Consulting Firms Exploring Large Language Models" for practical insights into the field. Additionally, explore Nature\'s overview of large language models to grasp their core concepts and applications.\n\nFor evaluating LLMs, consider reading FrugalTesting\'s "Best Practices and Metrics for Evaluating Large Language Models (LLMs)" article to gain a comprehensive understanding of evaluation benchmarks and metrics used in the industry. Furthermore, Tredence\'s gui

In [ ]:
1. Understanding the basics: Start by gaining knowledge on machine learning (ML) concepts such as neural networks, deep learning, training algorithms,
optimization techniques, etc., to have a solid foundation for studying large language models.
2. Research papers and surveys: Read research papers and survey articles about large language models (LLMs), intelligent agents using LLMs, and their applications
in various domains like software engineering, human-agent interaction, and NLP tasks. Some key sources are available at the provided links: https://habr.com/ru/companies/bothub/, https://nerdit.ru/llm-agents/, https://habr.com/ru/companies/gaz-is/articles/884410/.
3. Online courses and tutorials: Enroll in online courses on platforms like Coursera, edX, or fast.ai to learn more about large language models and related topics such as deep learning frameworks (TensorFlow, PyTorch), natural language processing (NLP) techniques, and reinforcement learning for intelligent agents.
4. Implementing LLM projects: Work on open-source projects that involve using LLMs to build your own NLP models or agent systems. This can help you gain hands-on experience in applying these concepts to solve real-world problems.
5. Participate in forums and communities: Engage with online forums like Reddit's Machine Learning subreddit, GitHub repositories related to LLM projects, a
nd NLP research groups on LinkedIn or ResearchGate. Collaborating with others can provide valuable insights, guidance, and feedback.
6. Stay updated with the latest developments: Follow relevant blogs, newsletters, and social media accounts of leading organizations working in the field of AI,
such as Google's DeepMind, Microsoft's GitHub repository, or Facebook AI Research (FAIR). This will help you stay informed about new advancements.
